In [97]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [98]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [110]:
def build_network(width, height, depth, classes):
    input_layer = Input(shape=(width, height, depth))

    x = Conv2D(filters=32,
               kernel_size=(3,3),
               padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32,
               kernel_size=(3,3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(rate=0.25)(x)\
    
    x = Conv2D(filters=64,
               kernel_size=(3,3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64,
               kernel_size=(3,3),
               padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(rate=0.25)(x)

    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(rate=0.5)(x)

    x = Dense(units=classes)(x)
    output = Softmax()(x)

    return Model(input_layer, output)

In [100]:
def plot_model_history(model_history, metric, plot_name):
    plt.style.use('seaborn-darkgrid')
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)
    plt.title(f'{metric.upper()}')
    plt.ylim([0, 1])
    plt.savefig(f'{plot_name}.png')
    plt.close()

In [101]:
def load_image_and_label(image_path, target_size=(64,64)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, np.float32)
    image = tf.image.resize(image, target_size)

    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == CLASSES)
    label = tf.dtypes.cast(label, tf.float32)
    
    return image, label

In [102]:
def augment(image, label):
    image = tf.image.resize_with_crop_or_pad(image, 74, 74)
    image = tf.image.random_srop(image, size=(64,64,3))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)

    return image, label

In [103]:
def prepare_dataset(data_pattern):
    return (tf.data.Dataset
            .from_tensor_slices(data_pattern)
            .map(load_image_and_label,
                 num_parallel_calls=AUTOTUNE))

In [104]:
SEED = 999
np.random.seed(SEED)

In [105]:
base_path = ('C:/Users/hp/Documents/DATA/caltech-101/101_ObjectCategories/*/*.jpg')
image_pattern = str(base_path)
image_paths = [*glob.glob(image_pattern)]
image_paths = [p for p in image_paths if
               p.split(os.path.sep)[-2] !=
               'BACKGROUND_Google']

In [106]:
CLASSES = np.unique([p.split(os.path.sep)[-2]
                     for p in image_paths])

In [107]:
train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=SEED)

In [108]:
BATCH_SIZE = 64
BUFFER_SIZE = 1024
train_dataset = (prepare_dataset(train_paths)
                 .batch(BATCH_SIZE)
                 .shuffle(buffer_size=BUFFER_SIZE)
                 .prefetch(buffer_size=BUFFER_SIZE))
test_dataset = (prepare_dataset(test_paths)
                .batch(BATCH_SIZE)
                .prefetch(buffer_size=BUFFER_SIZE))


In [113]:
EPOCHS = 10
model = build_network(64, 64, 3, len(CLASSES))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=EPOCHS)
result = model.evaluate(test_dataset)
print(f"test accuracy: {result[1]}")
plot_model_history(history, 'accuracy', 'normal')

Epoch 1/10
